In [ ]:
pip install langchain langchain-community langchain-text-splitters langchain-huggingface langchain-groq chromadb sentence-transformers unstructured


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

GROQ_API_KEY = "your_api_key"
PERSIST_DIR = "./chroma_db"
URL = "https://en.wikipedia.org/wiki/San_Francisco"

# 1. Load and prepare documents
loader = UnstructuredURLLoader(urls=[URL])
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = Chroma(persist_directory=PERSIST_DIR, embedding_function=embeddings)
vectordb.add_documents(chunks)
vectordb.persist()

retriever = vectordb.as_retriever(search_kwargs={"k": 3})

llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model="qwen/qwen3-32b",
    temperature=0,
)

# ---- Corrective RAG ----

# Step A: Judge relevance
judge_prompt = PromptTemplate(
    template="""
You are an evaluator.
Given a question and retrieved context, decide if the context is sufficient
to answer the question. Respond with "YES" or "NO" only.

Question: {question}
Context: {context}
""",
    input_variables=["question", "context"],
)
judge_chain = LLMChain(llm=llm, prompt=judge_prompt)

# Step B: Normal answer
answer_prompt = PromptTemplate(
    template="""
Use the following context to answer the question.

Context:
{context}

Question: {question}

Answer clearly and concisely:
""",
    input_variables=["context", "question"],
)
answer_chain = LLMChain(llm=llm, prompt=answer_prompt)

# Step C: Corrective answer (when context is poor)
corrective_prompt = PromptTemplate(
    template="""
The retrieved context is insufficient or irrelevant.
Do your best to answer the question using your own knowledge,
but also point out the missing context.

Question: {question}

Answer carefully:
""",
    input_variables=["question"],
)
corrective_chain = LLMChain(llm=llm, prompt=corrective_prompt)

# ---- Run ----
query = "Tell me about the climate of San Francisco."

retrieved_docs = retriever.get_relevant_documents(query)
context = "\n".join([d.page_content for d in retrieved_docs])

# Step A: Judge
judge_result = judge_chain.run({"question": query, "context": context}).strip()

# ---- Print the context that was judged ----
print("\n--- Context Passed to Judge ---\n")
print(context[:1000] + ("..." if len(context) > 1000 else ""))  # limit long output

print("\n--- Judge ---\n")
print(judge_result)

# Step B/C
if judge_result.upper().startswith("YES"):
    final_answer = answer_chain.run({"question": query, "context": context})
else:
    final_answer = corrective_chain.run({"question": query})

print("\n--- Final Answer ---\n")
print(final_answer)



--- Context Passed to Judge ---

Please set a user-agent and respect our robot policy https://w.wiki/4wJS. See also T400119.
Please set a user-agent and respect our robot policy https://w.wiki/4wJS. See also T400119.

--- Judge ---

<think>
Okay, let's see. The user is asking about the climate of San Francisco. The context provided has two lines that mention setting a user-agent and respecting a robot policy, with a link and a reference to T400119. Hmm, that doesn't seem related to climate at all. The context is probably some boilerplate text from a website's robots.txt or similar, warning bots to follow their policies. There's no actual information about San Francisco's climate here. So, the answer should be "NO" because the context doesn't provide any relevant data to answer the question.
</think>

NO

--- Final Answer ---

<think>
Okay, the user is asking about the climate of San Francisco. Let me start by recalling what I know. San Francisco is on the West Coast of the United Stat